In [1]:
import numpy as np
import pandas as pd
import re

## Exercise 1
Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

# Exercise 2
For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.

# Exercise 3
Find the lenders that have funded at least twice.

# Exercise 4
For each country, compute how many loans have involved that country as borrowers.

# Exercise 5
For each country, compute the overall amount of money borrowed.


# Exercise 6
Like the previous point, but expressed as a percentage of the overall amount lent.


# Exercise 7
Like the three previous points, but split for each year (with respect to disburse time).


# Exercise 8
For each lender, compute the overall amount of money lent.

# Exercise 9
For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

# Exercise 10
Which country has the highest ratio between the difference computed at the previous point and the population?

# Exercise 11
Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

# Exercise 12
For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

# Exercise 13
For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state

# Exercise 14
What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?

# Exercise 15
Cluster the loans according to the year-month of disburse time.

# Exercise 16
For each country, compute its overall GDP, by multiplying the per capita GDP with its population.

# Exercise 17
Find the country with the highest rate of irregular repayment interval.

# Exercise 18
Find the country with the highest fraction of loaned amount with irregular repayment interval.

# Exercise 19
For each lender, estimate the total amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.